In [1]:
import numpy as np
import networkx as nx
from scipy.linalg import sqrtm
import time

import perceval as pcvl
from perceval.algorithm import Sampler
import json
from  perceval.utils.postselect import PostSelect

In [2]:
#Generate inputstate from col_idx and total_modes
def prep_input(col_idx:list,total_modes:int):    
    #Use col_idx to prepare inputstate_raw (i.e inputstate as a list)
    inputstate_raw = [0 for i in range(total_modes)]
    
    for item in col_idx:
        inputstate_raw[item] = 1 #1 photon for the index position inside col_idx
    
    inputstate = pcvl.BasicState(inputstate_raw)
    return inputstate

#Condition builder
def condition_builder(desired_modes, undesired_modes, max_photon_per_mode = 1):
        #Builds conditions for perceval's PostSelect
        #desired_modes and undesired_modes are mode numbers mentioned in the list
        tempstr = ''
        first = True

        #desired
        ctr = 0
        for item in desired_modes:
            if first == True:
                if isinstance(max_photon_per_mode, int):
                    tempstr += '[' + str(item) + '] < ' + str(max_photon_per_mode + 1)
                    first = False
                else:
                    tempstr += '[' + str(item) + '] < ' + str(max_photon_per_mode[ctr] + 1)
                    first = False
                    ctr += 1
            else:
                if isinstance(max_photon_per_mode,int):
                    tempstr += ' & '
                    tempstr += '[' + str(item) + '] < ' + str(max_photon_per_mode + 1)
                else:
                    tempstr += ' & '
                    tempstr += '[' + str(item) + '] < ' + str(max_photon_per_mode[ctr] + 1)
                    ctr += 1
                    
        
        #undesired
        for item in undesired_modes:
            if first == True:
                tempstr += '[' + str(item) + '] == 0 '
                first = False
            else:
                tempstr += ' & '
                tempstr += '[' + str(item) + '] == 0'

        return tempstr

def get_probability(samples:list,max_modes,outputstate=None,row_postselect=False,max_photon_per_mode=1):
        #Only to be done AFTER boson sampling
        #row_postselect will only calculate the denominator for the starting m rows of U
        #max_photon_per_mode (ONLY TO BE USED WITH row_postselect currently) is going to post-select for that value 
        outputstate_str = str(pcvl.BasicState(outputstate)) #outputstate
        
        if row_postselect == False:
            total_count = len(samples['results'])
        else:
            total_count = 0 #we'll add to this total_count later on
            
            #get postselector ready
            desired_modes = [i for i in range(int(max_modes/2))]
            undesired_modes = [i for i in range(int(max_modes/2),max_modes)]
            print("Desired modes:",desired_modes)
            print("Undesired modes:",undesired_modes)
            
            condition_string = condition_builder(desired_modes,undesired_modes,max_photon_per_mode)
            ps = PostSelect(condition_string) #ps will come in handy in the next section

        output_count = 0
        
        for item in samples['results']:
            if outputstate_str == str(item):
                output_count += 1
            if row_postselect == True:
                if ps(pcvl.BasicState(str(item))) == True:
                    total_count += 1
            

        return output_count/total_count, output_count,total_count # prob, numerator, denominator

def get_probability2(samples:list,max_modes,max_photon_per_mode=1):
    #get_probability2 allows for multiple photons in the input
        
    total_count = 0 #we'll add to this total_count later on
            
    #get postselector ready
    desired_modes = [i for i in range(int(max_modes/2))]
    undesired_modes = [i for i in range(int(max_modes/2),max_modes)]
    print("Desired modes:",desired_modes)
    print("Undesired modes:",undesired_modes)
    
    #for denominator
    condition_string = condition_builder(desired_modes,undesired_modes,max(max_photon_per_mode))
    ps_denominator = PostSelect(condition_string) #ps will come in handy in the next section

    #for numerator
    #we are okay with desired modes having multiple photons
    condition_string = condition_builder(desired_modes,undesired_modes,max_photon_per_mode)
    ps_numerator = PostSelect(condition_string) #ps will come in handy in the next section

    no_of_rows = []
    

    output_count = 0

    for item in samples['results']:
        if ps_numerator(pcvl.BasicState(str(item))) == True:
            output_count += 1
            no_of_rows.append(np.count_nonzero(list(pcvl.BasicState(str(item)))))
            
        if ps_denominator(pcvl.BasicState(str(item))) == True:
            total_count += 1
    median_rows = np.median(no_of_rows)#calculate the median number of rows (where more than one photon landed)

    return output_count/total_count, output_count,total_count,median_rows # prob, numerator, denominator, median number of rows
    

In [3]:
#batch_no = 0 is with 10^5 shots
#batch_no = 1 is with 10^6 shots

In [5]:

desired_output_state = [0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0] + [0 for i in range(12)]
col_idx = [i for i in range(3,9)]
inputstate = prep_input(col_idx,24)

for batch_no in [0,1]:
    print("batch number:",batch_no)
    for i in range(0,6):
        filepath = './results/'+'res_bs_exp1_part'+str(i)+'_batch'+str(batch_no)+'.json'
        with open(filepath, 'r') as f:
            samples = json.load(f)
    
        print("Desired probability (basic):",get_probability(samples,24,desired_output_state,row_postselect=False))
        print("Desired probability (postselected):",get_probability(samples,24,desired_output_state,row_postselect=True))
    
        print("Desired probability (postselected/acceptable threshold)",get_probability2(samples,24,[0,0,0,3,3,3,3,3,3,0,0,0]))
        
    
        print("----")
    print("************")


batch number: 0
Desired probability (basic): (0.00022, 22, 100000)
Desired modes: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Undesired modes: [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
Desired probability (postselected): (0.88, 22, 25)
Desired modes: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Undesired modes: [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
Desired probability (postselected/acceptable threshold) (0.977645305514158, 1312, 1342, 4.0)
----
Desired probability (basic): (0.00017, 17, 100000)
Desired modes: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Undesired modes: [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
Desired probability (postselected): (0.5483870967741935, 17, 31)
Desired modes: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Undesired modes: [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
Desired probability (postselected/acceptable threshold) (0.9154030327214685, 1147, 1253, 4.0)
----
Desired probability (basic): (0.00022, 22, 100000)
Desired modes: [0, 1, 2, 3, 4, 5, 6, 7

In [10]:
max_modes = 24
desired_modes = [i for i in range(int(max_modes/2))]
undesired_modes = [i for i in range(int(max_modes/2),max_modes)]
condition_builder(desired_modes, undesired_modes, max_photon_per_mode = [0,0,0,3,3,3,3,3,3,0,0,0])

'[0] < 1 & [1] < 1 & [2] < 1 & [3] < 4 & [4] < 4 & [5] < 4 & [6] < 4 & [7] < 4 & [8] < 4 & [9] < 1 & [10] < 1 & [11] < 1 & [12] == 0 & [13] == 0 & [14] == 0 & [15] == 0 & [16] == 0 & [17] == 0 & [18] == 0 & [19] == 0 & [20] == 0 & [21] == 0 & [22] == 0 & [23] == 0'

In [39]:
len([0,0,0,3,3,3,3,3,3,0,0,0])

12

In [2]:
max([0,0,0,3,3,3,3,3,3,0,0,0])

3